# Power Outage

**Name(s)**: Luke, Andrew

**Website Link**: (your website link)

## Code

In [61]:
import pandas as pd
import numpy as np
import os
from scipy.stats import ks_2samp

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Cleaning and EDA

In [62]:
outage = pd.read_excel("outage.xlsx", sheet_name="Masterdata")
outage.head().to_clipboard()

In [63]:
# Drop Info Rows 
df = outage.drop(range(4)).dropna(axis=1, how='all')

# Set column names
df.columns = df.iloc[0]

# Drop Unit row and variables
df = df.drop([4, 5])
df = df.drop(columns="variables")

# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME'
df['OUTAGE.START'] = pd.to_datetime(df['OUTAGE.START.DATE']) + pd.to_timedelta(df['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME'
df['OUTAGE.RESTORATION'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(df['OUTAGE.RESTORATION.TIME'].astype(str))

# Drop the original date and time columns
df = df.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis=1)

# Fill Missing 
df.replace("NA", np.nan, inplace=True)

# Display 
df[['RES.SALES']]

df.to_csv("outage.csv")

In [64]:
len(df["variables"][df["variables"].isna()])

KeyError: 'variables'

### Assessment of Missingness

In [65]:
# I believe the column CUSTOMERS.AFFECTED is likely to be NMAR as if there are 
# fewer customers it is possible there is less incentive or ability to report 
# the count. Possible data that could help make it MAR would be data collection
# method as areas with less people might have inferior methods of data 
# collection. 

df = pd.read_csv('outage.csv')

def ks_query(missing, dependent):
    mar = df.copy()
    mar['missing'] = mar[missing].isna()
    res = ks_2samp(mar.query('missing')[dependent], mar.query('not missing')[dependent])
    return res

dur_vs_cust = ks_query('OUTAGE.DURATION', 'CUSTOMERS.AFFECTED')
dur_vs_sales = ks_query('OUTAGE.DURATION', 'TOTAL.SALES')

duration_missing = df.copy()
duration_missing['missing'] = duration_missing['OUTAGE.DURATION'].isna()

px.histogram(duration_missing, x='CUSTOMERS.AFFECTED', color='missing', histnorm='probability', marginal='box',
             title="customers affected by missingness of outage duration", barmode='overlay', opacity=0.7)

In [66]:
px.histogram(duration_missing, x='TOTAL.SALES', color='missing', histnorm='probability', marginal='box',
             title="total sales by missingness of outage duration", barmode='overlay', opacity=0.7)

### Hypothesis Testing

In [ ]:
# TODO